In [1]:
import os
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from langchain_core.tools import tool
from utils.google_calendar import get_available_slots, crear_evento
from langchain_core.runnables import RunnableConfig

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']
tavily_api_key = os.environ['TAVILY_API_KEY']
anthropic_api_key = os.environ['ANTHROPIC_API_KEY']

llm bootcamp
check!


In [2]:
@tool
def get_slots_tool()->list[tuple]:
    """tool that calls function utils.google_calendar.get_available_slots. It returns a list containing 3 tuples. In each tuple of len(2) there are
      two datetimes that refer to the start time and end time of the slot  """
    slots = get_available_slots()
    return slots

@tool
def crear_evento(start_time, end_time):
    """tool that calls function defined in utils.google_calendar.crear_event. It's purpose is to schedule a meeting in google calendar given start_time
    and end_time. Returns a boolean value, True if it was possible to schedule the meeting in the given time"""
    event = crear_evento(start_time, end_time)
    if event:
        return True
    else:
        return False
    


## helper functions

In [3]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda

from langgraph.prebuilt import ToolNode


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }

def create_tool_node_with_fallback(tools:list) -> dict:
    return ToolNode(tool).with_fallbacks(
        [RunnableLambda(handle_tool_error)]
    )

## Zero shot agent

When building, it's best to start with the simplest working implementation and use an evaluation tool like LangSmith to measure its efficacy. All else equal, prefer simple, scalable solutions to complicated ones. In this case, the single-graph approach has limitations. The bot may take undesired actions without user confirmation, struggle with complex queries, and lack focus in its responses. We'll address these issues later.

In this section, we will define a simple Zero-shot agent as the assistant, give the agent all of our tools, and prompt it to use them judiciously to assist the user.

The simple 2-node graph will look like the following:

![título de la imagen](images/part-1-diagram.png)

Define the state

In [4]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

Start with Agent

In [10]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig


class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        